In [16]:
import os
import pandas as pd
import torch
import pickle
import time
from torch.utils.data import DataLoader, ConcatDataset
from typing import Dict, List, Tuple
import simbench

In [17]:
from ResampleGAN.core.TrainerFactory import TrainerFactory
from ResampleGAN.core.TrainingConfig import TrainingConfig
from ResampleGAN.core.TrainingUtils import TrainingUtils, quick_setup
from ResampleGAN.utils.DatasetGenerator import DatasetGenerator

In [18]:
seed = 101
TrainingUtils.set_seed(seed)

In [19]:
now = f"Seed_{seed}"
base_dir = f"../results/005_all/reform/{now}"
for phase in [1, 2, 3]:
    phase_names = {1: "generator", 2: "discriminator", 3: "generator"}
    os.makedirs(f"{base_dir}/{phase}_{phase_names[phase]}", exist_ok=True)

In [20]:
logger, device = quick_setup(seed=seed, log_file="joint_training.log")
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

training_strategy = "skip"  # "skip" or "overwrite"
attention_types = {
    # "self": [3, 0, 0],
    # "conv": [0, 3, 0],
    "self+conv": [3, 3, 0],
    "self_conv": [3, 3, 0]
}
waveforms = ["electric", "pv", "wind", "mpv"]

2025-06-30 15:52:57,357 - INFO - Using device: cuda
2025-06-30 15:52:57,358 - INFO - GPU: NVIDIA RTX 3500 Ada Generation Laptop GPU
2025-06-30 15:52:57,358 - INFO - Memory: 12.0 GB


In [21]:
def process_waveform(waveform, batch_size):
    k, profile = waveform.split("_", 1)
    if k == "p":
        df = Enet.profiles["load"][[profile+"_pload"]].copy()
    elif k == "q":
        df = Enet.profiles["load"][[profile+"_qload"]].copy()
    elif k == "pv":
        df = Enet.profiles["renewables"][[profile]].copy()
    else:
        raise ValueError("type error")

    df_output = df.copy()
    df_output["time"] = pd.date_range(df.index[0], periods=len(df), freq="15min")
    df_output = df_output.set_index("time")
    df_input = df_output.resample("1h").first().ffill()
    dataset = DatasetGenerator(
        df_input=df_input,
        df_output=df_output,
        input_length=25,
        output_length=97,
        s_in="1h",
        s_out="15min",
        use_window=True
    )
    train_dataset, test_dataset,_ = DatasetGenerator.split_dataset(dataset, train_ratio=0.6, test_ratio=0.4, valid_ratio=0.0, shuffle=False)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [22]:
code = '1-LV-rural1--1-sw'
Enet = simbench.get_simbench_net(code)
Enet.sgen = pd.concat([Enet.sgen, Enet.sgen.iloc[0:4]], ignore_index=True)
Enet.sgen.loc[8, "name"] = "LV1.101 SGen 9"
Enet.sgen.loc[9, "name"] = "LV1.101 SGen 10"
Enet.sgen.loc[10, "name"] = "LV1.101 SGen 11"
Enet.sgen.loc[11, "name"] = "LV1.101 SGen 12"

Enet.sgen.loc[8, "bus"] = 12
Enet.sgen.loc[9, "bus"] = 14
Enet.sgen.loc[10, "bus"] = 6
Enet.sgen.loc[11, "bus"] = 9
# 2. Dimensionierung der Batteriespeicher beibehalten
Enet.storage["max_p_mw"] = -Enet.storage["min_p_mw"]

Enet.ext_grid['vm_pu'] = 1

C:\Workspace\Code\resamplegan\.venv\Lib\site-packages\simbench\converter\csv_pp_converter.py:874: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_data[output_name] = pd.concat([output_data[output_name], input_data[
C:\Workspace\Code\resamplegan\.venv\Lib\site-packages\simbench\converter\csv_pp_converter.py:874: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_data[output_name] = pd.concat([output_data[output_name], input_data[
C:\Workspace\Code\resamplegan\.venv\Lib\site-packages\simben

In [23]:
profiles_load_p = []
for i,profile in enumerate(Enet.load.profile.values):
    profiles_load_p.append(Enet.profiles["load"][profile+"_pload"].values[96:]*Enet.load["p_mw"][i])

profiles_load_q = []
for i,profile in enumerate(Enet.load.profile.values):
    profiles_load_q.append(Enet.profiles["load"][profile+"_qload"].values[96:]*Enet.load["q_mvar"][i])
#     profiles_load_q.append(Enet.profiles["load"][profile+"_qload"].values*0)
profiles_pv_p = []
for i,profile in enumerate(Enet.sgen.profile.values):
    profiles_pv_p.append(Enet.profiles["renewables"][profile].values[96:]*Enet.sgen["p_mw"][i])

In [24]:
data = {"p":Enet.load.profile.drop_duplicates().values, "q":Enet.load.profile.drop_duplicates().values, "pv":Enet.sgen.profile.drop_duplicates().values}

In [25]:
waveforms = [f"{key}_{value}" for key, values in data.items() for value in values]

In [26]:
def create_phase_config(phase: int, waveform: str, key: str, attention: List[int]) -> TrainingConfig:
    """Create training configuration for specified phase"""

    # Set attention type
    if key == "self_conv":
        attention_type = [["original"]*attention[0], ["conv"]*attention[1], ["freq"]*attention[2]]
    else:
        attention_type = ["original"]*attention[0] + ["conv"]*attention[1] + ["freq"]*attention[2]

    # Adjust batch size
    batch_size = 8 if waveform == "wind" else 16

    # Set weights based on phase
    if phase == 1:
        weights = {
            'mse': 1,
            'smoothness': 1,
            'gradient': 1,
        }
    else:  # phase 2 and 3
        weights = {
            'mse': 1,
            'feature_space': 1,
            'smoothness': 1,
            'gradient': 1,
        }

    return TrainingConfig(
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        n_epochs=1,
        batch_size=batch_size,
        lr=2e-4,
        weight_decay=0.01,
        grad_clip_threshold=10,
        dim_input=1,
        dim_attention=128,
        num_heads=4,
        dim_feedforward=128,
        dropout=0.1,
        num_layers=6,
        attention_type=attention_type,
        with_bias=False,
        weights=weights,
        optimizer_type='AdamW',
        scheduler_type='WarmupCosine',
        lambda_gan=0.1,
        critic=1,
        use_early_stopping=False,
        patience=10
    )

In [27]:
def train_single_phase_experiment(phase: int, waveform: str, key: str, attention: List[int], now: str, training_strategy: str, logger) -> bool:
    """
    Simplified function for training single phase experiment

    Returns:
        bool: Whether training was successful
    """

    # Check if should skip
    if training_strategy != "overwrite":
        phase_names = {1: "generator", 2: "discriminator", 3: "generator"}
        model_path = f"../results/002_all/reform/{now}/{phase}_{phase_names[phase]}/best_generator_{key}_{waveform}_self_{attention[0]}_conv_{attention[1]}_freq_{attention[2]}.pth"
        if os.path.exists(model_path):
            logger.info(f"⏭️ Skip existing model: Phase {phase} - {key}_{waveform}")
            return True

    start_time = time.time()
    logger.info(f"Start training Phase {phase}: {key}_{waveform} - {attention}")

    # Create configuration
    config = create_phase_config(phase, waveform, key, attention)

    # Create data loaders
    train_loader, test_loader = process_waveform(waveform, config.batch_size)

    # Set save directory
    phase_names = {1: "generator", 2: "discriminator", 3: "generator"}
    save_dir = f"../results/002_all/reform/{now}/{phase}_{phase_names[phase]}"
    base_save_dir = f"../results/002_all/reform/{now}"

    # Create trainer
    trainer = TrainerFactory.create_trainer(phase, config, save_dir, logger, key, attention, waveform, now, base_save_dir)

    # Start training
    history = trainer.train(train_loader, test_loader)

    # Save training history
    history_filename = f"losses_{key}_{waveform}_self_{attention[0]}_conv_{attention[1]}_freq_{attention[2]}.pkl"
    history_path = os.path.join(save_dir, history_filename)
    with open(history_path, "wb") as f:
        pickle.dump(history, f)

    end_time = time.time()
    execution_time = end_time - start_time
    logger.info(f"Phase {phase} - {key}_{waveform} training completed, time taken: {execution_time:.4f} seconds")

    return True

In [28]:
# Cell 6: Phase 1 - Generator Pretraining
def run_phase1():
    """Execute Phase 1 training"""
    logger.info("="*60)
    logger.info("Starting Phase 1: Generator Pretraining")
    logger.info("="*60)

    success_count = 0
    total_experiments = len(waveforms) * len(attention_types)

    for waveform in waveforms:
        logger.info(f"Processing waveform: {waveform}")

        for key, attention in attention_types.items():
            success = train_single_phase_experiment(1, waveform, key, attention, now, training_strategy, logger)
            if success:
                success_count += 1

    logger.info(f"Phase 1 completed! Success rate: {success_count}/{total_experiments}")
    return success_count, total_experiments

phase1_success, phase1_total = run_phase1()

2025-06-30 15:52:59,958 - INFO - ============================================================
2025-06-30 15:52:59,958 - INFO - Starting Phase 1: Generator Pretraining
2025-06-30 15:52:59,958 - INFO - ============================================================
2025-06-30 15:52:59,959 - INFO - Processing waveform: p_L2-A
2025-06-30 15:52:59,959 - INFO - ⏭️ Skip existing model: Phase 1 - self+conv_p_L2-A
2025-06-30 15:52:59,960 - INFO - ⏭️ Skip existing model: Phase 1 - self_conv_p_L2-A
2025-06-30 15:52:59,960 - INFO - Processing waveform: p_H0-C
2025-06-30 15:52:59,960 - INFO - ⏭️ Skip existing model: Phase 1 - self+conv_p_H0-C
2025-06-30 15:52:59,961 - INFO - ⏭️ Skip existing model: Phase 1 - self_conv_p_H0-C
2025-06-30 15:52:59,962 - INFO - Processing waveform: p_L1-A
2025-06-30 15:52:59,962 - INFO - ⏭️ Skip existing model: Phase 1 - self+conv_p_L1-A
2025-06-30 15:52:59,963 - INFO - ⏭️ Skip existing model: Phase 1 - self_conv_p_L1-A
2025-06-30 15:52:59,964 - INFO - Processing waveform

In [29]:
# Cell 7: Phase 2 - Joint Training
def run_phase2():
    """Execute Phase 2 training"""
    logger.info("="*60)
    logger.info("Starting Phase 2: Joint Training")
    logger.info("="*60)

    success_count = 0
    total_experiments = len(waveforms) * len(attention_types)

    for waveform in waveforms:
        logger.info(f"Processing waveform: {waveform}")

        for key, attention in attention_types.items():
            success = train_single_phase_experiment(2, waveform, key, attention, now, training_strategy, logger)
            if success:
                success_count += 1

    logger.info(f"Phase 2 completed! Success rate: {success_count}/{total_experiments}")
    return success_count, total_experiments

# Execute Phase 2
phase2_success, phase2_total = run_phase2()

2025-06-30 15:53:00,019 - INFO - ============================================================
2025-06-30 15:53:00,019 - INFO - Starting Phase 2: Joint Training
2025-06-30 15:53:00,020 - INFO - ============================================================
2025-06-30 15:53:00,020 - INFO - Processing waveform: p_L2-A
2025-06-30 15:53:00,020 - INFO - ⏭️ Skip existing model: Phase 2 - self+conv_p_L2-A
2025-06-30 15:53:00,021 - INFO - ⏭️ Skip existing model: Phase 2 - self_conv_p_L2-A
2025-06-30 15:53:00,022 - INFO - Processing waveform: p_H0-C
2025-06-30 15:53:00,022 - INFO - ⏭️ Skip existing model: Phase 2 - self+conv_p_H0-C
2025-06-30 15:53:00,023 - INFO - ⏭️ Skip existing model: Phase 2 - self_conv_p_H0-C
2025-06-30 15:53:00,023 - INFO - Processing waveform: p_L1-A
2025-06-30 15:53:00,024 - INFO - ⏭️ Skip existing model: Phase 2 - self+conv_p_L1-A
2025-06-30 15:53:00,024 - INFO - ⏭️ Skip existing model: Phase 2 - self_conv_p_L1-A
2025-06-30 15:53:00,025 - INFO - Processing waveform: p_H0-

In [30]:
def run_phase3():
    """Execute Phase 3 training"""
    logger.info("="*60)
    logger.info("Starting Phase 3: Generator Fine-tuning")
    logger.info("="*60)

    success_count = 0
    total_experiments = len(waveforms) * len(attention_types)

    for waveform in waveforms:
        logger.info(f"Processing waveform: {waveform}")

        for key, attention in attention_types.items():
            success = train_single_phase_experiment(3, waveform, key, attention, now, training_strategy, logger)
            if success:
                success_count += 1

    logger.info(f"Phase 3 completed! Success rate: {success_count}/{total_experiments}")
    return success_count, total_experiments

# Execute Phase 3
phase3_success, phase3_total = run_phase3()

2025-06-30 15:53:00,065 - INFO - ============================================================
2025-06-30 15:53:00,066 - INFO - Starting Phase 3: Generator Fine-tuning
2025-06-30 15:53:00,066 - INFO - ============================================================
2025-06-30 15:53:00,066 - INFO - Processing waveform: p_L2-A
2025-06-30 15:53:00,067 - INFO - ⏭️ Skip existing model: Phase 3 - self+conv_p_L2-A
2025-06-30 15:53:00,067 - INFO - ⏭️ Skip existing model: Phase 3 - self_conv_p_L2-A
2025-06-30 15:53:00,068 - INFO - Processing waveform: p_H0-C
2025-06-30 15:53:00,068 - INFO - ⏭️ Skip existing model: Phase 3 - self+conv_p_H0-C
2025-06-30 15:53:00,068 - INFO - ⏭️ Skip existing model: Phase 3 - self_conv_p_H0-C
2025-06-30 15:53:00,068 - INFO - Processing waveform: p_L1-A
2025-06-30 15:53:00,069 - INFO - ⏭️ Skip existing model: Phase 3 - self+conv_p_L1-A
2025-06-30 15:53:00,070 - INFO - ⏭️ Skip existing model: Phase 3 - self_conv_p_L1-A
2025-06-30 15:53:00,070 - INFO - Processing waveform